In [1]:
import pickle
from data_utils import load_graph, load_queries_by_formula

In [36]:
embed_dim = 128
data_dir = "./data/BIO/processed/"

Original data

In [37]:
rels, adj_lists, node_maps = pickle.load(open(data_dir+"/graph_data.pkl", "rb"))

In [29]:
rels 
# dict{type_from: [(type_to, rel_type), (type_to, rel_type), (type_to, rel_type), ...]}

{'proxy': [('aggregation', 'http://wwwopenarchivesorg/ore/terms/proxyIn'),
  ('physicalthing', 'http://wwwopenarchivesorg/ore/terms/proxyFor'),
  ('proxy', 'http://purlorg/collections/nl/am/partsReference'),
  ('proxy', 'http://purlorg/collections/nl/am/partOfReference'),
  ('agent', 'http://purlorg/collections/nl/am/contentPersonName'),
  ('agent', 'http://purlorg/collections/nl/am/associationPerson'),
  ('proxy', 'http://purlorg/collections/nl/am/relatedObjectReference')],
 'aggregation': [('physicalthing',
   'http://wwweuropeanaeu/schemas/edm/aggregatedCHO'),
  ('webresource', 'http://wwweuropeanaeu/schemas/edm/object'),
  ('webresource', 'http://wwweuropeanaeu/schemas/edm/landingPage'),
  ('proxy', 'http://wwwopenarchivesorg/ore/terms/proxyIn'),
  ('physicalthing', 'http://wwwopenarchivesorg/ore/terms/aggregates')],
 'physicalthing': [('aggregation',
   'http://wwweuropeanaeu/schemas/edm/aggregatedCHO'),
  ('aggregation', 'http://wwwopenarchivesorg/ore/terms/aggregates'),
  ('prox

In [30]:
adj_lists 
# dict {rel: {id_from: {id_to, id_to, id_to}}}
# where rel = (type_from, rel_type, type_to)

{('proxy',
  'http://wwwopenarchivesorg/ore/terms/proxyIn',
  'aggregation'): defaultdict(set,
             {24715: {25487},
              244908: {42972},
              310272: {151030},
              340308: {100619},
              122389: {122390},
              135068: {33944},
              173370: {156526},
              247158: {31319},
              161589: {244373},
              251954: {251243},
              298948: {273907},
              21794: {21795},
              7111: {83108},
              322563: {122074},
              366: {156785},
              229875: {153519},
              117847: {117848},
              147072: {147073},
              46265: {46266},
              7519: {11325},
              24551: {69711},
              239491: {104250},
              7460: {156103},
              216272: {212395},
              267041: {216839},
              273342: {139236},
              95385: {87476},
              47081: {47082},
              11099: {11100},
     

In [6]:
node_maps
# dict{node_type: [node_id, node_id, node_id]}
# can be used to map from type_id to global_id

{'publication': [1,
  3,
  4,
  7,
  8,
  10,
  13,
  14,
  16,
  19,
  22,
  24,
  28,
  30,
  32,
  34,
  36,
  38,
  40,
  41,
  42,
  45,
  47,
  49,
  50,
  52,
  54,
  56,
  58,
  60,
  61,
  64,
  66,
  68,
  69,
  71,
  72,
  74,
  75,
  78,
  79,
  81,
  82,
  85,
  86,
  89,
  91,
  93,
  94,
  95,
  99,
  101,
  104,
  105,
  107,
  111,
  112,
  118,
  119,
  126,
  127,
  128,
  129,
  131,
  133,
  134,
  137,
  139,
  141,
  143,
  144,
  147,
  149,
  150,
  152,
  153,
  154,
  155,
  162,
  163,
  164,
  165,
  166,
  167,
  168,
  170,
  172,
  175,
  177,
  180,
  182,
  183,
  184,
  186,
  188,
  191,
  193,
  195,
  196,
  199,
  200,
  204,
  205,
  208,
  209,
  210,
  211,
  213,
  214,
  215,
  217,
  219,
  220,
  222,
  224,
  227,
  228,
  230,
  233,
  234,
  238,
  242,
  244,
  245,
  249,
  251,
  253,
  254,
  255,
  260,
  261,
  262,
  263,
  264,
  265,
  269,
  270,
  271,
  272,
  273,
  275,
  276,
  278,
  280,
  283,
  285,
  286,
  289,
  290

In [7]:
# total number of nodes
sum([len(node_maps[key]) for key in node_maps.keys()])

2601

In [8]:
# check unique global entity ids
ents = []
for key in node_maps.keys():
    ents += node_maps[key]
print(len(set(ents)))

2601


Load graph

In [38]:
graph, feature_modules, node_maps = load_graph(data_dir, embed_dim)

In [10]:
# graph attributes
graph.__dict__.keys()

dict_keys(['features', 'feature_dims', 'relations', 'adj_lists', 'full_sets', 'full_lists', 'meta_neighs', 'edges', 'rel_edges', 'rel_weights', 'mode_edges', 'mode_weights', 'flat_adj_lists'])

In [11]:
graph.feature_dims

{'publication': 128,
 'person': 128,
 'class': 128,
 'organization': 128,
 'topic': 128,
 'project': 128}

In [39]:
# see all relations as list of  (from, reltype, to)
all_rels = []
for fr in list(graph.relations.keys()):
    for to_r in graph.relations[fr]:
        to, r = to_r
        all_rels.append((fr, r, to))

print("Unique relationships:",len(all_rels))
for rel in all_rels:
    print(rel)

Unique relationships: 56
('function', '0', 'function')
('function', '0', 'protein')
('drug', 'developmental_disorder_of_mental_health', 'drug')
('drug', 'sleep_disorder', 'drug')
('drug', 'cognitive_disorder', 'drug')
('drug', 'fungal_infectious_disease', 'drug')
('drug', 'urinary_system_disease', 'drug')
('drug', 'cardiovascular_system_disease', 'drug')
('drug', 'chromosomal_disease', 'drug')
('drug', 'thoracic_disease', 'drug')
('drug', '0', 'disease')
('drug', 'irritable_bowel_syndrome', 'drug')
('drug', 'monogenic_disease', 'drug')
('drug', 'sexual_disorder', 'drug')
('drug', 'nervous_system_disease', 'drug')
('drug', 'cryptorchidism', 'drug')
('drug', 'polycystic_ovary_syndrome', 'drug')
('drug', 'acquired_metabolic_disease', 'drug')
('drug', 'somatoform_disorder', 'drug')
('drug', 'hypospadias', 'drug')
('drug', 'integumentary_system_disease', 'drug')
('drug', 'viral_infectious_disease', 'drug')
('drug', 'bacterial_infectious_disease', 'drug')
('drug', 'personality_disorder', 'dr

In [13]:
# check if there are inversed edges -> all graphs contain symmetric edges!
num_inversed = 0
for rel in all_rels:
    fr, r, to = rel
    inversed = (to, r, fr)
    if inversed in all_rels:
        num_inversed += 1
        print(rel)
        print(inversed)
        print("")

print("Total number:", num_inversed)

('publication', 'http://swrcontowareorg/ontology#author', 'person')
('person', 'http://swrcontowareorg/ontology#author', 'publication')

('publication', 'http://wwww3org/1999/02/22-rdf-syntax-ns#type', 'class')
('class', 'http://wwww3org/1999/02/22-rdf-syntax-ns#type', 'publication')

('publication', 'http://swrcontowareorg/ontology#publishes', 'organization')
('organization', 'http://swrcontowareorg/ontology#publishes', 'publication')

('publication', 'http://swrcontowareorg/ontology#hasProject', 'project')
('project', 'http://swrcontowareorg/ontology#hasProject', 'publication')

('publication', 'http://swrcontowareorg/ontology#publication', 'person')
('person', 'http://swrcontowareorg/ontology#publication', 'publication')

('publication', 'http://swrcontowareorg/ontology#projectInfo', 'project')
('project', 'http://swrcontowareorg/ontology#projectInfo', 'publication')

('publication', 'http://swrcontowareorg/ontology#editor', 'person')
('person', 'http://swrcontowareorg/ontology#edit

In [14]:
feature_modules

{'publication': Embedding(1233, 128),
 'person': Embedding(1059, 128),
 'class': Embedding(55, 128),
 'organization': Embedding(34, 128),
 'topic': Embedding(147, 128),
 'project': Embedding(79, 128)}

In [15]:
node_maps
# provide mapping from global ID -> type-specific entity ID
# used for getting embedding!

tensor([  0,   0,   1,  ..., 520, 519,  -1])

In [16]:
# this one contains +1 nodes!
len(node_maps)

2602

Queries

In [17]:
train_queries = load_queries_by_formula(data_dir + "/train_edges.pkl")
for i in range(2, 4):
    train_queries.update(load_queries_by_formula(data_dir + "/train_queries_{:d}.pkl".format(i)))

In [18]:
# number of different query 'formulas' -> not possible to hardcode!
formulas = []
structures = list(train_queries.keys())
for key in structures:
    for form in train_queries[key]:
        formulas.append(form)

print(len(formulas))

9756


In [24]:
chainqueries = train_queries["3-inter_chain"]
first_formula = list(chainqueries.keys())[0]
formqueries = chainqueries[first_formula]
query = formqueries[0]

In [25]:
query.__dict__

{'formula': <graph.Formula at 0x27001958148>,
 'anchor_nodes': (1813, 117),
 'target_node': 117,
 'query_graph': None,
 'neg_samples': [29],
 'hard_neg_samples': [604]}

In [26]:
query.formula.__dict__

{'query_type': '3-inter_chain',
 'target_mode': 'project',
 'rels': (('project', 'http://swrcontowareorg/ontology#isAbout', 'topic'),
  (('project', 'http://swrcontowareorg/ontology#isAbout', 'topic'),
   ('topic', 'http://swrcontowareorg/ontology#dealtWithIn', 'project'))),
 'anchor_modes': ('topic', 'project')}

In [22]:
unique_nodes = []
query_types = list(train_queries.keys())
for key in query_types:
    query_list = train_queries[key]
    for formula in list(query_list.keys()):
        queries = query_list[formula]
        for query in queries:
            nodes = list(query.anchor_nodes)
            types = list(query.formula.anchor_modes)
            for n, t in zip(nodes, types):
                if t == 'class':
                    unique_nodes.append(n)
final = sorted(set(unique_nodes))
print(len(final))
final
# there are more embeddings than that there are unique nodes in query!

48


[23,
 27,
 67,
 80,
 98,
 103,
 157,
 158,
 300,
 437,
 475,
 476,
 645,
 646,
 726,
 766,
 785,
 959,
 1138,
 1440,
 1461,
 1609,
 1610,
 1710,
 1959,
 2053,
 2056,
 2062,
 2086,
 2087,
 2145,
 2154,
 2282,
 2356,
 2357,
 2369,
 2378,
 2423,
 2462,
 2486,
 2526,
 2533,
 2560,
 2562,
 2564,
 2569,
 2578,
 2579]

In [23]:
mapped = []
for n in final:
    mapped.append(n)
mapped.sort()
print("Len:",len(mapped))
mapped
# node_maps map the graph_id to the entity_type id!
# node IDs in the query are global and not graph-specific.
# They need a mapping if you want to use dict{type: embedding} framework...

Len: 48


[23,
 27,
 67,
 80,
 98,
 103,
 157,
 158,
 300,
 437,
 475,
 476,
 645,
 646,
 726,
 766,
 785,
 959,
 1138,
 1440,
 1461,
 1609,
 1610,
 1710,
 1959,
 2053,
 2056,
 2062,
 2086,
 2087,
 2145,
 2154,
 2282,
 2356,
 2357,
 2369,
 2378,
 2423,
 2462,
 2486,
 2526,
 2533,
 2560,
 2562,
 2564,
 2569,
 2578,
 2579]